# Données

## Données et Matrice

On considère qu'une donnée $x_i$ est décrite par $m$ caractéristiques réelles. La donnée $x_i$ est ainsi assimilable à un vecteur $x_i = (x_i^1, \dots, x_i^m)$ de $\mathbb{R}^m$. Si l'on possède $n$ données, on peut les représenter sous la forme d'une matrice :

$$X = \left(
\begin{array}{cccccc}
x^1_1&\dots &x^j_1 &\dots &x_1^m\\
     &      &\vdots&      &  \\
x_i^1&\dots &x^j_i &\dots &x_i^m\\
     &      &\vdots&      & \\
x_n^1&\dots &x^j_n&\dots &x_n^m
\end{array}
\right)$$

Où :

* une donnée est un vecteur ligne $x_i$ à $m$ coordonnées
* un caractère est un vecteur colonne  $x^j$  à $n$ coordonnées

> Nous nous restreingons à des donées réelles, mais il existe tout un tas d'autres type de données (catégorielles, entière, booléennes, ...) qui mobilisent chacunes leurs propres méthodes d'analyse.

C'est ce modèle de donnée qui est utilisé dans la bibliothèque [`pandas`](https://pandas.pydata.org/). 

* un jeu de donné est un objet du type [`pandas.DataFrame`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)
* chaque caractéristique (les colonnes d'un `DataFrame`) est un objet de type [`pandas.Series`](https://pandas.pydata.org/docs/reference/api/pandas.Series.html)

Manipuler et analyser des données en pandas revient à utiliser des méthodes de l'objet `pandas.DataFrame` ou de `pandas.Series`.

## Jeu de données

Le jeu de donné que l'on va utiliser est ici le résultat d'un contrôle (fichier "*epreuve.txt*"). Il y a 26 étudiants et pour chacun sont listés le temps mit pour faire l'épreuve et le nombre d'erreurs.

**On regarde** le fichier dans un éditeur de texte et on voit que :

* l'encodage du fichier est : unicode
* c'est un fichier csv dont le délimiteur est un espace

### Importation avec `pandas`

In [ ]:
import pandas

In [ ]:
épreuve = pandas.read_csv("./épreuve.txt", delim_whitespace=True)
épreuve

### Représentation graphique

Comme nos données sont des vecteurs de $\mathbb{R}^2$ on peut les repréenter dans le plan.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set()

> **Attention !** On a appelé `sns` la bibliothèque seaborn. Une fonction de la documentation nommé `seaborn.nom_méthode` devra être appelée `sns.nom_méthode` ici.

On représente les données comme des points en 2 dimensions. Pour cela on utilise [`seaborn.scatterplot`](https://seaborn.pydata.org/generated/seaborn.scatterplot.html) :

In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))

sns.scatterplot(x=épreuve['temps'], 
                y=épreuve['erreurs'],
                ax=ax)
for i in épreuve.index:
    ax.text(épreuve['temps'][i], épreuve['erreurs'][i], str(i))
plt.show()

PLusiuers étudiants ont pris le même temps et fait le même nombre d'erreurs. Pour essayer de voir tous les étudiants, on peut faire comme pour des données catégorielles, c'est à dire ajouter un *jitter* en x.  Mais attention, ce ne sont plus les données originelles représentées comme des point d'un espace vectoriel.

On utilise alors [`seaborn.stripplot`](https://seaborn.pydata.org/generated/seaborn.stripplot.html) :

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
sns.stripplot(x=épreuve['temps'], 
              y=épreuve['erreurs'], 
              ax=ax)
plt.show()

Pour les labels, c'est un peu plus compliqué. Ci-dessous une solution possible.

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
sns.stripplot(x=épreuve['temps'], 
              y=épreuve['erreurs'], 
              ax=ax
             )
temps_values = épreuve['temps'].drop_duplicates().sort_values().to_list()
décalage = {}
for i in épreuve.index:
    x, y = temps_values.index(épreuve['temps'][i]), épreuve['erreurs'][i]
    if (x, y) not in décalage:
        décalage[(x, y)] = 0
    else:
        décalage[(x, y)] += .1
    ax.text(x + décalage[(x, y)], y, str(i))

plt.show()

# Résumer des données

Lorsque le jeu de donnée devient trop grand pour l'appréhender directement, on a coutume de le résumer par des valeurs qui auront chacune pour tâche d'éclairer un pan des données.

## Grouper des données

Ici on va étudier chaque caractère indépendamment.

### Histogrammes

On utilise un histogramme, directement présent pour un dataframe pandas ([`pandas.DataFrame.hist`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.hist.html)) : 

> **Attention !** Nos données sont ici des dataframes, on peut donc directement utiliser `df.hist` si nos données sont appelées `df`.

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5), ncols=2) # 2 histogrammes, un sur chaque colonne
épreuve.hist(ax=ax)
plt.show()

### Densités

On supperpose à un histogramme un estimateur de densité. Ceci peut se faire avec la méthode [`seaborn.histplot`](https://seaborn.pydata.org/generated/seaborn.histplot.html) de seaborn :

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
sns.histplot(data=épreuve, x='temps', kde=True, ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
sns.histplot(data=épreuve, x='erreurs', kde=True, ax=ax)
plt.show()

## Statistiques descriptives

Il n'y a presque jamais de raisons de résumer une donnée par un nombre. En revanche résumer un caractère a beaucoup de sens car il permet de voir comment une donée particulière se situe par rapport aux autres données.

On utilise les statistiques descriptives que sont :

* la moyenne
* variance et écart-type
* les [quartiles](https://fr.wikipedia.org/wiki/Quartile)
* min et max

Ces données sont directement accessibles avec pandas si le dataframe est numérique (ce qui est le cas ici) avec la méthode : [`pandas.DataFrame.describe`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html)

In [ ]:
épreuve.describe()

On peut représenter ces nombres dans [une boîte à moustache](https://en.wikipedia.org/wiki/Box_plot) avec la méthode [`pandas.DataFrame.boxplot`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.boxplot.html)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
sns.boxplot(data=épreuve, x='temps', ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
sns.boxplot(data=épreuve, x='erreurs', ax=ax)
plt.show()

Ou encore une [violin box](https://en.wikipedia.org/wiki/Violin_plot) qui représente la densité estimée avec [`seaborn.violinplot`](https://seaborn.pydata.org/generated/seaborn.violinplot.html)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
sns.violinplot(data=épreuve, x='temps', ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
sns.violinplot(data=épreuve, x='erreurs', ax=ax)
plt.show()

# Qu'est ce que la moyenne et la variance ?

## Valeur centrale d'ordre $p$

Soit $u = (u_1, \dots, u_n)$ un vecteur de $\mathbb{R}^n$ muni de la norme $L_p$ :

$$
|| u ||_p = \sqrt[{\large p}]{\frac{1}{n}\sum_{1 \leq i \leq n}(u_i)^p}
$$

On appelle ***valeur centrale d'ordre $p$*** la valeur $c_p(u)$ tel que le vecteur constant $(c_p(u), \dots, c_p(u))$ réalise le minimum de (avec $C$ l'ensemble des vecteurs constant à $n$ dimensions) :

$$||u - c_p(u)||_p = \min_{c \in C} ||u - c||_p$$

La valeur centrale d'ordre $p$ est donc le minimum de la fonction :

$$
f(x) = \sum_{1 \leq i \leq n}(u_i-x)^p
$$

> Les valeurs centrales sont de bons résumés d'un vecteur par un nombre.

## Moyenne

> On appelle ***[moyenne](https://fr.wikipedia.org/wiki/Moyenne)*** d'un vecteur $u = (u_1, \dots, u_n)$ sa valeur centrale d'ordre 2 et on la note $\overline{u}$

En effet, la valeur centrale d'ordre 2 doit minimiser la fonction $f(x)$  :
  
$$
f_2(x) = \sum_{1 \leq i \leq n}(u_i-c)^2
$$

Ce minimum est facile à trouver en minimsant la dérivée.

> la moyenne d'un vecteur $u = (u_1, \dots, u_n)$ est :

$$
\overline{u} = \frac{1}{n}\sum_{1 \leq i \leq n}u_i
$$

In [ ]:
moyenne = épreuve.mean()
print(moyenne)

Ajoutons la moyenne à notre représentation graphique (croix rouge) :

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
sns.scatterplot(x=épreuve['temps'], 
                y=épreuve['erreurs'],
                ax=ax)

# for i in épreuve.index:
#     ax.text(épreuve['temps'][i], épreuve['erreurs'][i], str(i))

ax.scatter(moyenne['temps'], moyenne['erreurs'], marker="+", color="red")
plt.show()

## Médiane

> On appelle ***[médiane](https://fr.wikipedia.org/wiki/M%C3%A9diane_(statistiques))*** d'un vecteur $u = (u_1, \dots, u_n)$ sa valeur centrale d'ordre 1 et on la note $me(u)$.

La valeur centrale d'ordre 1 minimise la fonction :

$$f(x) = \sum_{i=1}^{n} |u_i - y|$$

Et on trouve en raisonnant par cas que $me(u)$ est l'élément du milieu de la liste triée contenant les éléments de $u$.

In [ ]:
mediane = épreuve.median()
print(mediane)

Ajoutons la médiane à notre représentation graphique (étoile mauve) :

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
sns.scatterplot(x=épreuve['temps'], 
                y=épreuve['erreurs'],
                ax=ax)

# for i in épreuve.index:
#     ax.text(épreuve['temps'][i], épreuve['erreurs'][i], str(i))

ax.scatter(mediane['temps'], mediane['erreurs'], marker="*", color="purple")
ax.scatter(moyenne['temps'], moyenne['erreurs'], marker="+", color="red")

plt.show()

## Dispersion

Un paramère de dispersion permet de mesurer l'amplitude des variations autour d'une valeur centrale. On la calcule comme une erreur.

On appelle ***paramètre de dispersion d'ordre $p$*** la valeur $d_p(u)$ correspondant à la valeur :

$$d_p(u) = ||u - c_p(u)||_p$$

Où $c_p(u)$ est la valeur centrale d'ordre $p$.

> On appelle ***[écart-type](https://fr.wikipedia.org/wiki/%C3%89cart_type)*** le paramètre de dispersion d'ordre 2

On le note $\sigma(u)$ et il vaut :

$$
\sigma(u) = \sqrt{\frac{1}{n}\sum_{1 \leq i \leq n}(u_i - \overline{u})^2}
$$

> On appelle ***[variance](https://fr.wikipedia.org/wiki/Variance_(math%C3%A9matiques))*** le carré de l'écart-type et on le note $\sigma^2(u)$

In [ ]:
écart_type =  épreuve.std()

écart_type

# Centrer et réduire les données

Lorsque l'on étudie non plus juste un paramètre des données, mais que l'on cherche à comparer des données entre elles, il faut faire un peu attention car on cherche en analyse des données à prendre en compte tous les paramètres de façon équivalente dans ces comparaisons.

> L'usage est de ***centrer*** et ***réduire*** les données.

La ***moyenne*** d'un jeu de donnée $X$ sera alors le vecteur :

$$
\overline{X} = (\overline{x^1}, \dots, \overline{x^j}, \dots, \overline{x^m})
$$

Et sa ***variance*** : 

$$
\sigma^2(X) = (\sigma^2(x^1), \dots, \sigma^2(x^j), \dots, \sigma^2(x^m))
$$


## Centrer

En considérant nos données comme un nuage de points, l'origine du repère peut être très éloigné (des humains mesurés en $\mu$mètres par exemple. Pour rendre son statut d'origine à l'origine, on a alors coutume de ***centrer les les données*** : c'est à dire d'effectuer la translation pour chaque **colonne** :

$$x^j \leftarrow x^j - \overline{x^j}$$

> La moyenne de données centrée est 0

In [ ]:
épreuve_centrée = épreuve - moyenne

épreuve_centrée

In [ ]:
épreuve_centrée.mean()

Les données centrées permettent de voir ce qui est petit / grand par rapport à la moyenne en regardant uniquement le signe des données.

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
sns.scatterplot(x=épreuve_centrée['temps'], 
                y=épreuve_centrée['erreurs'],
                ax=ax)

for i in épreuve.index:
    ax.text(épreuve_centrée['temps'][i], épreuve_centrée['erreurs'][i], str(i))

ax.axvline(0)
ax.axhline(0)
plt.show()

## Réduire

Comparer les données entre elles se fait en calculant une distance entres elles. En reprenant la norme $L_2$, comparer deux données se fait alors selon la formule :

$$
d^2(x_i, x_{i'}) = \frac{1}{n}\sum_{1\leq j \leq m}(x_i^j - x_{i'}^j)^2
$$

Pour que cette comparaison soit efficace il faut :

* que les variation de chaque colonne contribuent de la même façn à la distance
* ne faut pas qu'une caractéristique écrase les autres dans le calcul de la distance


Pour savoir comment se comportent les objets par rapport à leurs coordonnées on a un problème d'unité : temps et nombre d'erreurs. Plus pertinent de regarder l'écart par rapport à la position moyenne.

L'erreur des nombres par rapport à la moyenne est appelé variance : $\frac{1}{n} \sum_{i=1}^{n} (x_i - \bar{x})^2 = \sigma(x)^2$.
De là si on divise chaque nombre par $\sigma$ (appelé l'écart type) on obtient $x'_i = \frac{x_i}{\sigma}$ et de là la variance de $x' = (x_1, \dots, x_n)$ vaut $1$ et est un nombre sans unité : on homogénise les différents axes.  


Répondre au deux préoccupations précédente peut se faire en divisant chaque caractérisitque par son écart type : 

$$x^j \leftarrow \frac{x^j}{\sigma(x^j)}$$

Cette opération s'appelle ***réduire les données***

> L'écart-type de données réduites vaut 1

In [ ]:
écart_type =  épreuve.std()

écart_type

In [ ]:
épreuve_réduit = épreuve / écart_type

épreuve_réduit

In [ ]:
épreuve_réduit.std()

### Données centrées et réduites

Les données centrée et réduite sont dite homogénéisées. On peut les comparer car ils ont la même unité (sans unité) et leur variantion est identique (variance = écart à la moyenne = 1).
Cette opération étant quasi-obligatoire pour tout jeu de données, une méthode toute faite existe.

In [ ]:
épreuve_centrée_réduite = (épreuve - épreuve.mean()) / épreuve.std()
épreuve_centrée_réduite

In [ ]:
épreuve_centrée_réduite.describe()

On peut maintenant trouver les étudiants ayant fait moins bien que la moyenne en temps et en erreurs.

Au delà de ±2 c'est un gros écrart.

Représentons graphiquement ces données :

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
sns.scatterplot(x=épreuve_centrée_réduite['temps'], 
                y=épreuve_centrée_réduite['erreurs'],
                ax=ax)

for i in épreuve_centrée_réduite.index:
    ax.text(épreuve_centrée_réduite['temps'][i], épreuve_centrée_réduite['erreurs'][i], str(i))

ax.axvline(0)
ax.axhline(0)
plt.show()

### Homogénéisation avec `sklearn`

La bibliothèque [`sklearn`](https://scikit-learn.org/stable/) est très utilisée en machine learning. Nous allons l'utiiser dansune optique d'analyse des données.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
épreuve_scaled_sklearn = scaler.fit_transform(pandas.DataFrame(épreuve, dtype='float')) # sinon warning de conversion

Le résultat est un tableau [`numpy.ndarray`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html) et pas un dataframe de pandas. 

In [ ]:
type(épreuve_scaled_sklearn)

Remédions à ça en recréeant un dataframe :

In [ ]:
épreuve_scaled = pandas.DataFrame(épreuve_scaled_sklearn, columns=épreuve.columns)
épreuve_scaled

In [ ]:
épreuve_scaled.describe()

> **Attention** : sklearn utilise la [variance corrigée](https://fr.khanacademy.org/math/be-4eme-secondaire2/x213a6fc6f6c9e122:statistiques-1/x213a6fc6f6c9e122:variance-et-ecart-type/v/review-and-intuition-why-we-divide-by-n-1-for-the-unbiased-sample-variance) qui divise par $n-1$ et non par $n$. 
> 
> La variance corrigée est un estimateur sas biais de la variance (qui se calcule bien avec \frac{1}{n}$). Nous n'en avons pas besoin en analyse des données car notre population c'est nos données, on a pas besoin d'estimer la variance, on l'a.